In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import os
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
import matplotlib.cm as cm
import time
import scipy.io as io

import skimage.measure
from keras.initializers import RandomNormal
from keras.callbacks import ModelCheckpoint
from keras.models import *
from keras.layers import *
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.models import model_from_json
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
np.random.seed(16)
tf.random.set_seed(16)
random.seed(16)

In [ ]:
Train_imgs = []
Train_labels = []

path_images_train = ".../shanghai299/images_train299/"
path_heads_train = ".../shanghai299/ground_truth/"



for img_name, heads_name in zip(sorted(os.listdir(path = path_images_train)), sorted(os.listdir(path = path_heads_train))):

    img = preprocess_input(cv2.cvtColor(cv2.imread(path_images_train + img_name), cv2.COLOR_BGR2RGB))
    Train_imgs.append(img)

    Train_labels.append(len(loadmat(path_heads_train + heads_name)['image_info'][0, 0][0, 0][0]))


Train_imgs = np.asarray(Train_imgs)
Train_labels = np.asarray(Train_labels)

In [ ]:
Test_imgs = []
Test_labels = []

path_images_test = ".../shanghai299/images_test299/"
path_heads_test = ".../shanghai299/ground_truth/"



for img_name, heads_name in zip(sorted(os.listdir(path = path_images_test)), sorted(os.listdir(path = path_heads_test))):

    img = preprocess_input(cv2.cvtColor(cv2.imread(path_images_test + img_name), cv2.COLOR_BGR2RGB))
    Test_imgs.append(img)

    Test_labels.append(len(loadmat(path_heads_test + heads_name)['image_info'][0, 0][0, 0][0]))


Test_imgs = np.asarray(Test_imgs)
Test_labels = np.asarray(Test_labels)

In [ ]:
# Model

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='linear')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6, decay=5e-4), loss="mae", metrics=["mae"])

# Data Augmentation
#datagen = ImageDataGenerator(
#    rotation_range=20,
#    width_shift_range=0.2,
#    height_shift_range=0.2,
#    shear_range=0.2,
#    zoom_range=0.2,
#    horizontal_flip=True,
#    fill_mode='nearest',
#    validation_split=0.2)

checkpoint_filepath = './inception_crowd_counting_best.weights.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_mae',
    mode='min',
    save_best_only=True,
    verbose=1)

#early_stopping = EarlyStopping(monitor='val_mae', patience=10, restore_best_weights=True)
#reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=5, min_lr=1e-6)

history = model.fit(Train_imgs, Train_labels,
                    batch_size=32, epochs=100,
                    validation_split=0.2,
                    # callbacks=[model_checkpoint_callback, early_stopping, reduce_lr]
                    callbacks=[model_checkpoint_callback])


model.save('inception_crowd_counting_final.weights.h5')

In [ ]:
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history.history['mae'], label='train_mae')
plt.plot(history.history['val_mae'], label='val_mae')
plt.title('Mean Absolute Error')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
def test_model(model, test_imgs, test_labels, file_parameters_path):
    model.load_weights(file_parameters_path)

    mae_error = 0
    mse_error = 0
    mape_error = 0


    N = len(Test_labels)
    for i, (x, y) in enumerate(zip(Test_imgs, Test_labels)):

        print(i)

        mae_error += np.abs(np.sum(model.predict(np.reshape(np.asarray(x),(1, x.shape[0], x.shape[1], x.shape[2])))) - y)
        mse_error += np.square(np.sum(model.predict(np.reshape(np.asarray(x),(1, x.shape[0], x.shape[1], x.shape[2])))) - y)
        mape_error += np.abs((np.sum(model.predict(np.reshape(np.asarray(x),(1, x.shape[0], x.shape[1], x.shape[2])))) - y) / y)* 100

    return mae_error/N, np.sqrt(mse_error/N), mape_error/N

mae, mse, mape  = test_model(model, Test_imgs, Test_labels, checkpoint_filepath)
print(f"Mean Absolute Error: {mae}, Mean Square Error: {mse}, Mean Absolute Percentage Error:{mape}")